# 12.04 - Selenium: Sitios con JavaScript

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio-Avanzado  
**Tiempo estimado:** 35 min

---

## ¿Qué aprenderás?

- Cuándo usar Selenium vs BeautifulSoup
- Configurar un navegador headless
- Localizar elementos y esperar a que carguen
- Interactuar con formularios y botones
- Extraer datos de páginas dinámicas

---

## 1. ¿Cuándo necesitas Selenium?

| Situación | requests + BS4 | Selenium |
|---|---|---|
| HTML estático | ✅ | innecesario |
| Datos cargados con JS | ❌ | ✅ |
| Necesitas hacer click | ❌ | ✅ |
| Formularios de login | ❌ | ✅ |
| Scroll infinito | ❌ | ✅ |
| Velocidad | rápido | lento |

**Regla:** usa `requests` + `BeautifulSoup` siempre que puedas. Solo pasa a Selenium cuando el contenido requiera ejecución de JavaScript.

In [ ]:
# pip install selenium webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


def create_driver(headless: bool = True) -> webdriver.Chrome:
    """
    Crea un driver de Chrome configurado.

    Parameters
    ----------
    headless : bool
        Si True, el navegador no abre ventana visible

    Returns
    -------
    webdriver.Chrome
        Driver listo para usar
    """
    options = Options()
    if headless:
        options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('user-agent=DataPortfolio/1.0 (educational)')

    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)


print("Driver configurado correctamente")

---

## 2. Localizar elementos

In [ ]:
# Las formas principales de localizar elementos en Selenium

# By.ID         → <div id="station-1">
# By.CLASS_NAME → <div class="station">
# By.CSS_SELECTOR → 'div.station h2.name'
# By.XPATH      → '//div[@class="station"]/h2'
# By.TAG_NAME   → 'table'

# Ejemplo de flujo completo
def scrape_with_selenium(url: str) -> list[dict]:
    """
    Scraping de una página dinámica con Selenium.

    Parameters
    ----------
    url : str
        URL a scrapear

    Returns
    -------
    list[dict]
        Lista de registros extraídos
    """
    driver = create_driver(headless=True)
    records = []

    try:
        driver.get(url)

        # Esperar a que un elemento esté presente (evita errores por JS lento)
        wait = WebDriverWait(driver, timeout=10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.station')))

        # Extraer elementos
        stations = driver.find_elements(By.CSS_SELECTOR, 'div.station')

        for station in stations:
            name = station.find_element(By.CLASS_NAME, 'name').text
            bikes = station.find_element(By.CLASS_NAME, 'bikes').text
            records.append({'name': name, 'bikes': int(bikes)})

    finally:
        driver.quit()  # Siempre cerrar el driver

    return records


print("Función de scraping con Selenium definida")

---

## 3. Esperas explícitas (la clave del scraping dinámico)

In [ ]:
# NUNCA uses time.sleep() fijo → es frágil y lento
# USA esperas explícitas que esperan la condición exacta

from selenium.webdriver.support import expected_conditions as EC

def wait_for_element(driver, selector: str, timeout: int = 10):
    """Espera a que un elemento esté visible antes de interactuar."""
    wait = WebDriverWait(driver, timeout)
    return wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, selector))
    )


# Condiciones más útiles
# EC.presence_of_element_located  → el elemento existe en el DOM
# EC.visibility_of_element_located → el elemento es visible
# EC.element_to_be_clickable       → se puede hacer click
# EC.text_to_be_present_in_element → el texto ya cargó
# EC.staleness_of(element)         → el elemento desapareció (útil tras click)

print("Estrategias de espera definidas")

---

## 4. Interactuar con formularios

In [ ]:
from selenium.webdriver.common.keys import Keys


def search_and_scrape(url: str, search_term: str, search_selector: str) -> str:
    """
    Abre una página, escribe en un buscador y extrae el HTML resultante.

    Parameters
    ----------
    url : str
        URL de la página
    search_term : str
        Texto a buscar
    search_selector : str
        CSS selector del input de búsqueda

    Returns
    -------
    str
        HTML de la página de resultados
    """
    driver = create_driver(headless=True)

    try:
        driver.get(url)

        # Encontrar el campo de búsqueda
        search_box = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, search_selector))
        )

        # Escribir y enviar
        search_box.clear()
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)

        # Esperar a que carguen los resultados
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
        )

        return driver.page_source

    finally:
        driver.quit()


print("Función de búsqueda con formulario definida")

---

## 5. Scroll infinito

In [ ]:
import time


def scroll_to_bottom(driver, pause: float = 1.5, max_scrolls: int = 10) -> None:
    """
    Hace scroll hasta el fondo de la página para cargar contenido dinámico.

    Parameters
    ----------
    driver : webdriver.Chrome
        Driver activo
    pause : float
        Segundos de espera entre scrolls
    max_scrolls : int
        Máximo de scrolls para evitar bucles infinitos
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    for scroll in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(pause)

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            print(f"Fin de página alcanzado en scroll {scroll + 1}")
            break

        last_height = new_height
        print(f"Scroll {scroll + 1}: altura {new_height}px")


print("Función de scroll infinito definida")

---

## Resumen

| Concepto | Código |
|---|---|
| Crear driver headless | `Options() + add_argument('--headless')` |
| Buscar elemento | `driver.find_element(By.CSS_SELECTOR, 'div.x')` |
| Espera explícita | `WebDriverWait(driver, 10).until(EC....)` |
| Escribir en input | `element.send_keys('texto')` |
| Click | `element.click()` |
| Scroll | `driver.execute_script("window.scrollTo(0, ...)" )` |
| Cerrar driver | `driver.quit()` (siempre en `finally`) |

**Siempre cierra el driver en un bloque `finally`** para evitar procesos de Chrome huérfanos.

---

## Ejercicio

Crea un scraper con Selenium que abra Google Maps, busque "BiciMAD Sol Madrid", espere a que cargue el resultado y extraiga el nombre y la dirección de la primera coincidencia.

In [ ]:
# Tu solución aquí

---

**Anterior:** [12.03 - Scraping Real Cases](./12_03_scraping_real_cases.ipynb)  
**Siguiente:** [12.05 - Ética y Buenas Prácticas](./12_05_ethics_and_best_practices.ipynb)